<a href="https://colab.research.google.com/github/ambika1us/Data_Extraction_and_Text_Analysis_for_Blackcoffer/blob/main/Ambika_Prasad_Data_Extraction_and_Text_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install newspaper3k

In [ ]:
#importing library
import os
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
from nltk.tokenize import RegexpTokenizer
import re
import string
import newspaper
from newspaper import Article
from pathlib import Path
from newspaper import Config


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#importing input file
df=pd.read_excel('/content/drive/MyDrive/Blackcoffer/Input.xlsx')[['URL_ID','URL']]

In [ ]:
df.head()

,URL_ID,URL
0,37,https://insights.blackcoffer.com/ai-in-healthc...
1,38,https://insights.blackcoffer.com/what-if-the-c...
2,39,https://insights.blackcoffer.com/what-jobs-wil...
3,40,https://insights.blackcoffer.com/will-machine-...
4,41,https://insights.blackcoffer.com/will-ai-repla...


In [ ]:
def remove_punc(dmf):
  global nopunc
  nopunc = [char for char in dmf if char not in string.punctuation]
  nopunc = ''.join(nopunc)
  return nopunc

In [ ]:
def remove_stpwd(nopunc):
  global clean_mess
  clean_mess = [word for word in nopunc.split() if word.lower() not in stpwrd]
  clean_mess=' '.join(clean_mess)
  return clean_mess

In [ ]:
def tokenize(clean_mess):
  global tokens
  tokens = word_tokenize(clean_mess.upper())
  return tokens

In [ ]:
def neg_words(tokens):
  global num_neg
  file = open('negdict.txt', 'r')
  neg_words = file.read().split()

  num_neg=0
  for n in range(len(tokens)):
    words=str(tokens[n])
    if words in neg_words:
      num_neg=num_neg+1
    return num_neg


In [ ]:
def pos_words(tokens):
  global num_pos
  file = open('posdict.txt', 'r')
  pos_words = file.read().split()
  num_pos=0
  for n in range(len(tokens)):
    words=str(tokens[n])
    if words in pos_words:
      num_pos=num_pos+1
    return num_pos

In [ ]:
def polarity_sc(num_pos, num_neg):
  global polarity
  polarity=(num_pos-num_neg)/((num_pos+num_neg)+0.000001)
  return polarity

In [ ]:
def subjectivity_sc(num_pos, num_neg,tokens):
  global subjectivity
  subjectivity=(num_pos+num_neg)/(len(tokens)+0.000001)
  return subjectivity

In [ ]:
def avgsent_len(dmf):
  global avg_sent_len
  text = dmf
  sent=sent_tokenize(dmf)
  avg_sent_len=(len(dmf.split()))/(len(sent))
  return avg_sent_len

In [ ]:
def complex_words_percent(dmf):
  global complex_wrd
  global complex_per
  complex_wrd=0
  wrd=dmf.split()
  for i in wrd:
    if len(i)>2:
      complex_wrd=complex_wrd+1

    complex_per=complex_wrd/len(wrd)
    return complex_per

In [ ]:
def fog_index(avg_sent_len,complex_per):
  global fogind
  fogind=0.4*(avg_sent_len+complex_per)
  return fogind

In [ ]:
def avg_wrds_in_sent(dmf):
  global avg_wrds_per_sent
  totwrd=len(dmf.split())
  totsent=(len(sent_tokenize(dmf)))
  avg_wrds_per_sent=totwrd/totsent

  return avg_wrds_per_sent

In [ ]:
def complex_wrd_count(dmf):
  global complex_wrds
  complex_wrds=0
  words=dmf.split()
  for i in words:
    if len(i)>2:
      complex_wrds=complex_wrds+1
  return complex_wrds

In [ ]:
def word_count(clean_mess):
  global wrd_cnt
  wrd_cnt=len((clean_mess.split()))
  return wrd_cnt

In [ ]:
def syllable_count(dmf):
  global syl_count
  text = dmf
  syl_count = 0
  vowels = "aeiou"
  for word in text.lower().split():
    print(word)
    if word[0] in vowels:
      syl_count += 1
      for index in range(1, len(word)):
        if word[index] in vowels and word[index - 1] not in vowels:
          syl_count += 1
        if word.endswith("ed") or word.endswith("es"):
          syl_count -= 1
    return syl_count

In [ ]:
def personal_pronoun(dmf):
  global personal_pro
  pronounRegex = re.compile(r'\bI\b|\bwe\b|\bWe\b|\bmy\b|\bMy\b|\bours\b|\bus\b')
  pps=pronounRegex.findall(dmf)
  personal_pro=len(pps)
  return personal_pro

In [ ]:
def avg_wrd_length(dmf):
  global avg_wrdlen
  alltxt=dmf
  wrd_cnt=0
  char_cnt=0
  for word in alltxt:
    if word!= ' ':
      char_cnt=wrd_cnt+1
    totalchar=char_cnt
  for word in alltxt.split():
    wrd_cnt=wrd_cnt+1

  totalwrds=wrd_cnt
  avg_wrdlen=totalchar/totalwrds
  return avg_wrdlen

In [ ]:
USER_AGENT = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.71 Safari/537.36'
config = Config()
config.browser_user_agent = USER_AGENT
config.request_timeout = 40

In [ ]:
for n in range(len(df)):
  urlm=df['URL'][n]
  article=Article(urlm, config=config)
  article.download()
  article.parse()
  art=article.title
  art2=article.text
  p = Path(str(df['URL_ID'][n])+'.txt')
  p=  open(str(df['URL_ID'][n])+'.txt', 'a',encoding='utf-8')
  p.write(art)
  p = open(str(df['URL_ID'][n])+'.txt', 'a',encoding='utf-8')
  p.write('\n \n')
  p.write(art2)
  p.close()

In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
stpwrd = nltk.corpus.stopwords.words('english')

In [ ]:
file2=open('/content/drive/MyDrive/Blackcoffer/StopWords/StopWords_Auditor.txt')
file2=file2.read().split('\n')

In [ ]:
stpwrd.extend(file2)

In [152]:
mst_dict1=pd.read_csv('/content/drive/MyDrive/Blackcoffer/MasterDictionary/negative-words.txt',encoding="ISO-8859-1")
mst_dict2=pd.read_csv('/content/drive/MyDrive/Blackcoffer/MasterDictionary/positive-words.txt',encoding="ISO-8859-1")

In [166]:
for n in range(len(mst_dict1)):
  word=str(mst_dict1)
  if word!='nan':
    p=open('negdict.txt','a')
    p.write(word +' ')

for n in range(86531):
  word=str(mst_dict2)
  if word!='nan':
    p=open('posdict.txt','a')
    p.write(word +' ')

In [167]:
alldata={ 'URL_ID':[],
          'URL':[],
          'POSITIVE SCORE':[],
          'NEGATIVE SCORE':[],
          'POLARITY SCORE':[],
          'SUBJECTIVITY SCORE':[],
          'AVG SENTENCE LENGTH':[],
          'PERCENTAGE OF COMPLEX WORDS':[],
          'FOG INDEX':[],
          'AVG NUMBER OF WORDS PER SENTENCE':[],
          'COMPLEX WORD COUNT':[],
          'WORD COUNT':[],
          'SYLLABLE PER WORD':[],
          'PERSONAL PRONOUNS':[],
          'AVG WORD LENGTH':[]
        }

In [168]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [169]:
for n in range(len(df)):
    file=open(str(df['URL_ID'][n])+'.txt', 'r',encoding='utf-8')
    dmf=file.read()
    l=df['URL'][n]
    alldata['URL'].append(l)
    remove_punc(dmf)
    remove_stpwd(nopunc)
    tokenize( clean_mess)
    neg_words(tokens)
    pos_words(tokens)
    polarity_sc(num_pos, num_neg)
    subjectivity_sc(num_pos, num_neg,tokens)
    avgsent_len(dmf)
    complex_words_percent(dmf)
    fog_index(avg_sent_len,complex_per)
    avg_wrds_in_sent(dmf)
    complex_wrd_count(dmf)
    word_count(clean_mess)
    syllable_count(dmf)
    personal_pronoun(dmf)
    avg_wrd_length(dmf)
    alldata['URL_ID'].append(n)
    alldata['NEGATIVE SCORE'].append(num_neg)
    alldata['POSITIVE SCORE'].append(num_pos)
    alldata['POLARITY SCORE'].append(polarity)
    alldata['SUBJECTIVITY SCORE'].append(subjectivity)
    alldata['AVG SENTENCE LENGTH'].append(avg_sent_len)
    alldata['PERCENTAGE OF COMPLEX WORDS'].append(complex_per)
    alldata['FOG INDEX'].append(fogind)
    alldata['AVG NUMBER OF WORDS PER SENTENCE'].append(avg_wrds_per_sent)
    alldata['COMPLEX WORD COUNT'].append(complex_wrds)
    alldata['WORD COUNT'].append(wrd_cnt)
    alldata['SYLLABLE PER WORD'].append(syl_count)
    alldata['PERSONAL PRONOUNS'].append(personal_pro)
    alldata['AVG WORD LENGTH'].append(avg_wrdlen)

ai
what
what
will
will
will
how
how
deep
how
how
how
how
future
how
how
all
evolution
how
environmental
how
what
how
how
what
impact
what
how
can
embedding
management
are
will
will
will
what
why
how
can
how
how
is
global
gender
how
how
impacts
impacts
impact
impact
changing
online
human
how
how
how
impacts
coronavirus
covid-19
lessons
estimating
estimating
travel
gaming
what
due
impact
contribution
how
how
lessons
covid-19:
how
how
coronavirus:
estimating
covid-19:
how
lessons
lessons
coronavirus:
why
impact
impact
global
impact
should
how
how
controversy
coronavirus:
coronavirus
what
marketing
continued
coronavirus
should
why
what
coronavirus,
industrial
covid-19
impact
impact
impact
blockchain
blockchain
the
big
business
challenges


In [170]:
exceldata=pd.DataFrame.from_dict(alldata,orient='index')
exceldata.transpose()

,URL_ID,URL,POSITIVE SCORE,NEGATIVE SCORE,POLARITY SCORE,SUBJECTIVITY SCORE,AVG SENTENCE LENGTH,PERCENTAGE OF COMPLEX WORDS,FOG INDEX,AVG NUMBER OF WORDS PER SENTENCE,COMPLEX WORD COUNT,WORD COUNT,SYLLABLE PER WORD,PERSONAL PRONOUNS,AVG WORD LENGTH
0,0,https://insights.blackcoffer.com/ai-in-healthc...,0,0,0.0,0.0,23.589118,0.0,9.435647,23.589118,10444,8051,1,7,0.00008
1,1,https://insights.blackcoffer.com/what-if-the-c...,0,0,0.0,0.0,17.707665,0.000101,7.083106,17.707665,7981,5425,0,42,0.000101
2,2,https://insights.blackcoffer.com/what-jobs-wil...,0,0,0.0,0.0,20.073826,0.000084,8.029564,20.073826,9570,7063,0,21,0.000084
3,3,https://insights.blackcoffer.com/will-machine-...,0,0,0.0,0.0,17.448949,0.000086,6.979614,17.448949,9059,6433,0,119,0.000086
4,4,https://insights.blackcoffer.com/will-ai-repla...,0,0,0.0,0.0,22.088448,0.000082,8.835412,22.088448,9899,7063,0,112,0.000082
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106,106,https://insights.blackcoffer.com/blockchain-fo...,0,0,0.0,0.0,18.693878,0.001092,7.477988,18.693878,743,544,0,9,0.001092
107,107,https://insights.blackcoffer.com/the-future-of...,0,0,0.0,0.0,22.652174,0.00064,9.061125,22.652174,1263,977,0,2,0.00064
108,108,https://insights.blackcoffer.com/big-data-anal...,0,0,0.0,0.0,17.590909,0.000861,7.036708,17.590909,909,696,0,2,0.000861
109,109,https://insights.blackcoffer.com/business-anal...,0,0,0.0,0.0,24.322581,0.001326,9.729563,24.322581,622,454,0,0,0.001326
